In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler, Subset
import pandas as pd
import numpy as np
import ijson
import h5py
import json

/usr/lib64/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
#data recovery

data = pd.read_csv('cleavage.csv')
data = data.drop(0)
data.head()

print(len(data))

179079


In [3]:
data_cleaned = []

#suppression of data without precise output
for i in range(len(data)):
    if data.iloc[i,0][0:3] != "CLE":
        data_cleaned.append(data.iloc[i])

print(len(data_cleaned))
print(data_cleaned[0])

86675
code                                                     A01.001
uniprot_acc                                               P00004
p1                                                            11
cleavage_evidence                                   experimental
cleavage_type                                  non-physiological
cleavage_notes                                               NaN
cleavage_ontology                                            NaN
cleavage_location                                            NaN
residue_range                                              2-105
cutdb                                                        NaN
Ref                         <%Hamuro %etal, 2008[20080325A077]%>
peptide_identification                                        MS
peptidase_identification                                     NaN
mernum                                                       NaN
Name: 1, dtype: object


In [4]:
list_enzymes = []
for i in range(1, len(data_cleaned)):
    list_enzymes.append(data_cleaned[i]["code"])
        
list_enzymes = list(set(list_enzymes))
print("length of list_enzymes: ", len(list_enzymes))
print("length of data after removing incomplete line: ", len(data_cleaned))
print("example of list_enzymes: ", list_enzymes[0:5])

length of list_enzymes:  1229
length of data after removing incomplete line:  86675
example of list_enzymes:  ['M41.013', 'C32.003', 'S9G.094', 'N09.001', 'M12.311']


In [5]:
list_proteins = []
for i in range(1, len(data_cleaned)):
    list_proteins.append(data_cleaned[i]["uniprot_acc"])
        
list_proteins = list(set(list_proteins))
print("length of list_proteins: ", len(list_proteins))
print("length of data after removing incomplete line: ", len(data_cleaned))
print("example of list_proteins: ", list_proteins[0:5])

length of list_proteins:  14274
length of data after removing incomplete line:  86675
example of list_proteins:  ['Q9WTU0', 'Q5BKY9', 'Q13586', 'P08218', 'Q43715']


In [6]:
#recuperation of data on each protein (GO and sequence here)

dic_protein = {}
for protein in list_proteins:
    dic_protein[protein] = {}

i = 0
with open('../uniprotkb_AND_reviewed_true_2024_03_26.json', "rb") as f:
    for record in ijson.items(f, "results.item"):
        try:
            i += 1
            refs = record.get("uniProtKBCrossReferences", [])
            if record["primaryAccession"] in list_proteins:
                GO = [ref["id"] for ref in refs if ref.get("database") == "GO"]
                sequence = record["sequence"]["value"]
                dic_protein[record["primaryAccession"]] = [GO, sequence]
                    
            if i % 10000 == 0:
                print(i)
                
        except Exception as record_error:
            print("Error processing record:", record_error)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000


In [7]:
#recuperation of data on each protein (embedding here)

i = 0
with h5py.File("../per-protein.h5", "r") as file:
    print(f"number of entries: {len(file.items())}")
    for sequence_id, embedding in file.items():
        i += 1
        if i % 10000 == 0:
            print(i)
        if sequence_id in dic_protein:
            dic_protein[sequence_id].append(np.array(embedding).tolist())

number of entries: 570820
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000


In [8]:
#count the number of each output to keep the enzyme for which we have enough data

count_enzyme_dic = {}

for enzyme in list_enzymes:
    count_enzyme_dic[enzyme] = 0

for i in range(0, len(data_cleaned)):
    count_enzyme_dic[data_cleaned[i]["code"]] += 1

In [9]:
X_cl = []   #cleavage_site
y_name = []   #name of enzyme
y_index = []   #index of the data
X_seq = []   #sequence of protein
X_embedding = []   #embedding of protein
X_GO = []   #GO of protein

j = 0

for i in range(0, len(data_cleaned)):
    enzyme = data_cleaned[i]["code"]
    protein = data_cleaned[i]["uniprot_acc"]
    if len(dic_protein[protein]) == 3 and count_enzyme_dic[enzyme] >= 5:    #all information about protein and more than 5 data for the enzyme
        sequence = dic_protein[protein][1]
        prot_embedding = dic_protein[protein][2]
        cleavage = int(data_cleaned[i]["p1"])
        go = dic_protein[protein][0]
        X_cl.append(cleavage)
        y_name.append(enzyme)
        y_index.append(j)
        X_seq.append(sequence)
        X_embedding.append(prot_embedding)
        X_GO.append(go)
        j += 1

In [10]:
#encoding of the go terms

count_go = {}

for i in range(len(X_GO)):
    for go in X_GO[i]:
        if go in count_go:
            count_go[go] += 1
        else:
            count_go[go] = 1

print("number of go terms: ", len(count_go))
print("example: ", list(count_go.items())[0:5])

#we keep the most frequent go terms
number_go = 2000

most_frequent_go = dict(sorted(count_go.items(), key=lambda item: item[1], reverse=True))

most_frequent_go = dict(list(most_frequent_go.items())[0:number_go])

X_GO_filtered = []

for i in range(len(X_GO)):
    go_filtered = []
    for go in X_GO[i]:
        if go in most_frequent_go:
            go_filtered.append(go)
    X_GO_filtered.append(go_filtered)

# encoder X_GO_filtered

list_go = list(most_frequent_go.keys())
dic_GO = {}
for i in range(len(list_go)):
    dic_GO[list_go[i]] = i

X_GO_filtered_int = []

for i in range(len(X_GO_filtered)):
    go = X_GO_filtered[i]
    go_int = [0]*len(list_go)
    for j in range(len(go)):
        go_int[dic_GO[go[j]]] = 1
    X_GO_filtered_int.append(go_int)

print(X_GO_filtered_int[0])
print(len(X_GO_filtered_int[0]))

#enregistrement of dic_GO

with open('dic_GO_problem_1.json', 'w') as fp:
    json.dump(dic_GO, fp)

number of go terms:  15411
example:  [('GO:0070069', 28), ('GO:0005829', 42159), ('GO:0005758', 1025), ('GO:0070469', 675), ('GO:0009055', 1275)]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
#encoding of the cleavage environement

window_size = 10

X_pep = []
for i in range(len(X_seq)):
    seq = X_seq[i]
    p1 = X_cl[i]
    pep = []
    min = p1-window_size
    max = p1+window_size
    for j in range(min,max):
        if j < 0:
            pep.append('X')
        elif j >= len(seq):
            pep.append('X')
        else:
            pep.append(seq[j])
    X_pep.append(pep)

#tranformation of peptides into vocab
vocab = ['A','B','C','D','E','F','G','H','I','K','L','M','N','0','P','Q','R','S','T','U','V','W','X','Y','Z']
vocab_dict = {}
for i in range(len(vocab)):
    vocab_dict[vocab[i]] = i

X_pep_int = []
for i in range(len(X_pep)):
    pep = X_pep[i]
    pep_int = []
    for j in range(len(pep)):
        pep_int.append(vocab_dict[pep[j]])
    X_pep_int.append(pep_int)

In [12]:
#encoding of enzyme names
list_enzymes = list(set(y_name))
dic_enzyme = {}
for i in range(len(list_enzymes)):
    dic_enzyme[list_enzymes[i]] = i

y_name_int = []
for i in range(len(y_name)):
    y_encoded = [0]*len(list_enzymes)
    y_encoded[dic_enzyme[y_name[i]]] = 1
    y_name_int.append(y_encoded)

#enregistrement of dic_enzyme

with open('dic_enzyme.json', 'w') as fp:
    json.dump(dic_enzyme, fp)

In [13]:
print(y_index[0])
print(X_pep_int[0])
print(X_seq[0])
print(X_cl[0])
print(y_name[0])
print(y_name_int[0])
print(X_embedding[0])
print(X_GO[0])

0
[6, 3, 20, 4, 9, 6, 9, 9, 8, 5, 20, 15, 9, 2, 0, 15, 2, 7, 18, 20]
MGDVEKGKKIFVQKCAQCHTVEKGGKHKTGPNLHGLFGRKTGQAPGFTYTDANKNKGITWKEETLMEYLENPKKYIPGTKMIFAGIKKKTEREDLIAYLKKATNE
11
A01.001
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_embedding_t = torch.tensor(X_embedding, dtype=torch.float32).to(device)
X_pep_t = torch.tensor(X_pep_int, dtype=torch.int64).to(device)
X_cl_t = torch.tensor(X_cl, dtype=torch.float32).to(device)
X_cl_t = X_cl_t.unsqueeze(1).to(device)
X_go_t = torch.tensor(X_GO_filtered_int, dtype=torch.float32).to(device)
y_t = torch.tensor(y_name_int, dtype=torch.float32).to(device)

In [15]:
class model_embedding_pep_site(nn.Module):
    def __init__(self, embed_dim, output_dim):
        super(model_embedding_pep_site, self).__init__()
        self.fc_embed = nn.Linear(embed_dim, 2048)
        self.embed = nn.Embedding(26, 128)
        self.gru = nn.GRU(128, 256, 2, batch_first=True)
        self.fc_pep = nn.Linear(256, 2048)
        self.fc_site = nn.Linear(1, 10)
        self.fc1 = nn.Linear(2048 * 2 + 10, 2048)
        self.fc2 = nn.Linear(2048, output_dim)
    
    def forward(self, embed, pep, site):
        x_embed = F.relu(self.fc_embed(embed))
        x_pep = self.embed(pep)
        x_pep, _ = self.gru(x_pep)
        x_pep = x_pep[:,-1,:]
        x_pep = F.relu(self.fc_pep(x_pep))
        x_site = F.relu(self.fc_site(site))
        x = torch.cat((x_embed, x_pep, x_site), dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

class model_embedding_pep_go_site(nn.Module):
    def __init__(self, embed_dim, go_dim, output_dim):
        super(model_embedding_pep_go_site, self).__init__()
        self.fc_embed = nn.Linear(embed_dim, 2048)
        self.embed = nn.Embedding(26, 128)
        self.gru = nn.GRU(128, 256, 2, batch_first=True)
        self.fc_pep = nn.Linear(256, 2048)
        self.fc_go = nn.Linear(go_dim, 2048)
        self.fc_site = nn.Linear(1, 10)
        self.fc1 = nn.Linear(2048 * 3 + 10, 2048)
        self.fc2 = nn.Linear(2048, output_dim)
    
    def forward(self, embed, pep, site, go):
        x_embed = F.relu(self.fc_embed(embed))
        x_pep = self.embed(pep)
        x_pep, _ = self.gru(x_pep)
        x_pep = x_pep[:,-1,:]
        x_pep = F.relu(self.fc_pep(x_pep))
        x_go = F.relu(self.fc_go(go))
        x_site = F.relu(self.fc_site(site))
        x = torch.cat((x_embed, x_pep, x_go, x_site), dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [16]:
def train_with_go(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        embed, pep, site, go, labels = data
        optimizer.zero_grad()
        outputs = model(embed, pep, site, go)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def test_accuracy_with_go(model, dataloader, criterion):
    model.eval()
    correct = 0
    correct3 = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for data in dataloader:
            embed, pep, site, go, labels = data
            outputs = model(embed, pep, site, go)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            #top 3 predictions
            _, predicted3 = torch.topk(outputs.data, 3, dim=1)
            for i in range(len(labels)):
                if labels[i] in predicted3[i]:
                    correct3 += 1
    return correct / total, correct3 / total, running_loss / len(dataloader)

In [18]:
X_embedding_train, X_embedding_test, X_pep_train, X_pep_test, X_cl_train, X_cl_test, X_go_train, X_go_test, y_train, y_test = train_test_split(X_embedding_t, X_pep_t, X_cl_t, X_go_t, y_t, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_embedding_train, X_pep_train, X_cl_train, X_go_train, y_train)
test_dataset = TensorDataset(X_embedding_test, X_pep_test, X_cl_test, X_go_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# parameters
embed_dim = X_embedding_t.shape[1]   #embedding dimension = 1024
go_dim = X_go_t.shape[1]   #go terms dimension = 2000
output_dim = y_t.shape[1]   #number of enzymes = 535

model = model_embedding_pep_go_site(embed_dim, go_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

# early stopping

n_epochs = 100
patience = 5
best_acc = 0

for epoch in range(n_epochs):
    train_loss = train_with_go(model, train_loader, optimizer, criterion)
    acc, acc3, test_loss = test_accuracy_with_go(model, test_loader, criterion)
    print(f"Epoch {epoch+1}/{n_epochs} | Train loss: {train_loss:.3f} | Test loss: {test_loss:.3f} | Test accuracy: {acc:.3f} | Test top 3 accuracy: {acc3:.3f}")
    if acc > best_acc:
        best_acc = acc
        patience_counter = 0
        torch.save(model.state_dict(), "model_embedding_pep_go_site_problem_1_final.pt")
    else:
        patience_counter += 1
    if patience_counter > patience:
        break

Epoch 1/100 | Train loss: 0.008 | Test loss: 0.006 | Test accuracy: 0.410 | Test top 3 accuracy: 0.597
Epoch 2/100 | Train loss: 0.005 | Test loss: 0.005 | Test accuracy: 0.460 | Test top 3 accuracy: 0.666
Epoch 3/100 | Train loss: 0.005 | Test loss: 0.005 | Test accuracy: 0.475 | Test top 3 accuracy: 0.696
Epoch 4/100 | Train loss: 0.004 | Test loss: 0.005 | Test accuracy: 0.489 | Test top 3 accuracy: 0.712
Epoch 5/100 | Train loss: 0.004 | Test loss: 0.005 | Test accuracy: 0.499 | Test top 3 accuracy: 0.720
Epoch 6/100 | Train loss: 0.004 | Test loss: 0.004 | Test accuracy: 0.516 | Test top 3 accuracy: 0.741
Epoch 7/100 | Train loss: 0.004 | Test loss: 0.004 | Test accuracy: 0.530 | Test top 3 accuracy: 0.757
Epoch 8/100 | Train loss: 0.003 | Test loss: 0.004 | Test accuracy: 0.537 | Test top 3 accuracy: 0.764
Epoch 9/100 | Train loss: 0.003 | Test loss: 0.004 | Test accuracy: 0.542 | Test top 3 accuracy: 0.767
Epoch 10/100 | Train loss: 0.003 | Test loss: 0.004 | Test accuracy: 0.55

In [19]:
def train_without_go(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        embed, pep, site, go, labels = data
        optimizer.zero_grad()
        outputs = model(embed, pep, site)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def test_accuracy_without_go(model, dataloader, criterion):
    model.eval()
    correct = 0
    correct3 = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for data in dataloader:
            embed, pep, site, go, labels = data
            outputs = model(embed, pep, site)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            #top 3 predictions
            _, predicted3 = torch.topk(outputs.data, 3, dim=1)
            for i in range(len(labels)):
                if labels[i] in predicted3[i]:
                    correct3 += 1
    return correct / total, correct3 / total, running_loss / len(dataloader)

In [20]:
X_embedding_train, X_embedding_test, X_pep_train, X_pep_test, X_cl_train, X_cl_test, X_go_train, X_go_test, y_train, y_test = train_test_split(X_embedding_t, X_pep_t, X_cl_t, X_go_t, y_t, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_embedding_train, X_pep_train, X_cl_train, X_go_train, y_train)
test_dataset = TensorDataset(X_embedding_test, X_pep_test, X_cl_test, X_go_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# parameters
embed_dim = X_embedding_t.shape[1]   #embedding dimension = 1024
go_dim = X_go_t.shape[1]   #go terms dimension = 2000
output_dim = y_t.shape[1]   #number of enzymes = 535

model = model_embedding_pep_site(embed_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

# early stopping

n_epochs = 100
patience = 5
best_acc = 0

for epoch in range(n_epochs):
    train_loss = train_without_go(model, train_loader, optimizer, criterion)
    acc, acc3, test_loss = test_accuracy_without_go(model, test_loader, criterion)
    print(f"Epoch {epoch+1}/{n_epochs} | Train loss: {train_loss:.3f} | Test loss: {test_loss:.3f} | Test accuracy: {acc:.3f} | Test top 3 accuracy: {acc3:.3f}")
    if acc > best_acc:
        best_acc = acc
        patience_counter = 0
        torch.save(model.state_dict(), "model_embedding_pep_site_problem_1_final.pt")
    else:
        patience_counter += 1
    if patience_counter > patience:
        break

Epoch 1/100 | Train loss: 0.008 | Test loss: 0.006 | Test accuracy: 0.390 | Test top 3 accuracy: 0.562
Epoch 2/100 | Train loss: 0.006 | Test loss: 0.006 | Test accuracy: 0.413 | Test top 3 accuracy: 0.613
Epoch 3/100 | Train loss: 0.005 | Test loss: 0.005 | Test accuracy: 0.443 | Test top 3 accuracy: 0.642
Epoch 4/100 | Train loss: 0.005 | Test loss: 0.005 | Test accuracy: 0.471 | Test top 3 accuracy: 0.670
Epoch 5/100 | Train loss: 0.005 | Test loss: 0.005 | Test accuracy: 0.487 | Test top 3 accuracy: 0.693
Epoch 6/100 | Train loss: 0.004 | Test loss: 0.005 | Test accuracy: 0.510 | Test top 3 accuracy: 0.719
Epoch 7/100 | Train loss: 0.004 | Test loss: 0.005 | Test accuracy: 0.519 | Test top 3 accuracy: 0.733
Epoch 8/100 | Train loss: 0.004 | Test loss: 0.004 | Test accuracy: 0.534 | Test top 3 accuracy: 0.747
Epoch 9/100 | Train loss: 0.004 | Test loss: 0.004 | Test accuracy: 0.536 | Test top 3 accuracy: 0.751
Epoch 10/100 | Train loss: 0.003 | Test loss: 0.004 | Test accuracy: 0.54